# Check sync process

In [1]:
from scripts.conf_file_finding import try_find_conf_file
try_find_conf_file()

Local configuration file found !!, no need to run the configuration (unless configuration has changed)


In [2]:
import datetime
import pathlib
import numpy as np
import pylab as pl
import matplotlib
import matplotlib.pyplot as plt
from scipy.signal import find_peaks


from u19_pipeline import acquisition, ephys
from u19_pipeline.ephys import ephys_element, probe_element, get_session_directory
import u19_pipeline.ephys as ephys
import u19_pipeline.acquisition as acquisition
import datajoint as dj

import u19_pipeline.utils.DemoReadSGLXData.readSGLX as readSGLX
import u19_pipeline.utils.ephys_utils as ephys_utils
from bitstring import BitArray

Connecting alvaros@datajoint00.pni.princeton.edu:3306


In [3]:
key = {
    'subject_fullname': 'testuser_test70',
    'session_date': datetime.date(2021, 10, 12)}


key = (acquisition.Session & key).fetch1('KEY')
key

{'subject_fullname': 'testuser_test70',
 'session_date': datetime.date(2021, 10, 12),
 'session_number': 0}

The module ephys_element contains all the tables designed in the DataJoint ephys element, we could draw the diagram to see the schema structures

# Synchronize ePhys and Behavior

Populate the tables, using the nidaq file produced by spikeGLX. This is done automatically and produces a record of VR iteration numbers measured in the time of the ePhys setup.

In [ ]:
ephys.BehaviorSync()

In [ ]:
#ephys.BehaviorSync.populate(key)

In [ ]:
(ephys.BehaviorSync & key).delete()

In [ ]:
ephys.EphysSession()

### Read nidaq file and behavior data

In [ ]:
session_dir = pathlib.Path(get_session_directory(key))
nidq_bin_full_path = list(session_dir.glob('*nidq.bin*'))[0]

#Nidaq file
nidq_meta          = readSGLX.readMeta(nidq_bin_full_path)
nidq_sampling_rate = readSGLX.SampRate(nidq_meta)
digital_array      = ephys_utils.spice_glx_utility.load_spice_glx_digital_file(nidq_bin_full_path, nidq_meta)

#Behavior data
behavior = dj.create_virtual_module('behavior', 'u19_behavior')
thissession = behavior.TowersBlock().Trial() & key
behavior_time, iterstart = thissession.fetch('trial_time', 'vi_start')


In [ ]:
digital_array.shape

### Get samples and times where trials and iterations start

In [ ]:
iteration_dict = ephys_utils.get_iteration_sample_vector_from_digital_lines_pulses(digital_array[1,:], digital_array[2,:], nidq_sampling_rate)
iteration_dict

### Assert number of trials and iterations

In [ ]:
status = ephys_utils.assert_iteration_samples_count(iteration_dict['iter_start_idx'], behavior_time)
status

### Plot time differences between behavior file and nidaq time

In [ ]:
x = np.array([])
for i in range(iteration_dict['iter_times_idx'].shape[0]):
    s = behavior_time[i].flatten()-iteration_dict['iter_times_idx'][i]
    x = np.append(x,s, axis=0)

plt.plot(x)

### Insert BehaviorSync record

In [ ]:
final_key = dict(key, nidq_sampling_rate = nidq_sampling_rate, 
               iteration_index_nidq = iteration_dict['framenumber_vector_samples'],
               trial_index_nidq = iteration_dict['trialnumber_vector_samples'])
final_key
ephys.BehaviorSync.insert1(final_key,allow_direct_insert=True)


Once the synchronization files have been produces, the all Clusters/spikes can be binned into iterations.

In [5]:
ephys.BehaviorSync.populate(key)

nChan: 1, nFileSamp: 1453278
7
7
1355
1355
2269
2269
4961
4961
2891
2891
1044
1044
2226
2226
2701
2701
True
{'subject_fullname': 'testuser_test70', 'session_date': datetime.date(2021, 10, 12), 'session_number': 0, 'nidq_sampling_rate': 5000.062709, 'iteration_index_nidq': array([nan, nan, nan, ..., nan, nan, nan]), 'trial_index_nidq': array([nan, nan, nan, ..., nan, nan, nan])}


In [6]:
ephys.BehaviorSync()

subject_fullname username_mouse_nickname,session_date date of experiment,session_number number,nidq_sampling_rate sampling rate of behavioral iterations niSampRate in nidq meta file,iteration_index_nidq length of this longblob should be the number of iterations in the behavior recording,trial_index_nidq length of this longblob should be the number of iterations in the behavior recording
hnieh_E105,2021-01-15,0,5000.08,=BLOB=,=BLOB=
ms81_M004,2021-05-07,0,5000.06,=BLOB=,=BLOB=
ms81_M005,2021-05-05,0,5000.07,=BLOB=,=BLOB=
testuser_test70,2021-10-12,0,5000.06,=BLOB=,=BLOB=
